In [ ]:
#Mount Google Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Install Necessary Packages for Colab
!pip install pyspark
!pip install graphframes
!apt-get install scala
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install networkx

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 65.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=921cae1e1ea7e8fd1ee1c183731b22ed165aeb7e33850a2131bb1da1e327b2de
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 154 kB 5.1 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java
  scala-library scala-parser-combinators scala-xml
Suggested packages:
  scala-doc
The following NEW packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java
  scala scala-library scala-parser-combinators scala-xml
0 upgraded, 8 newly installed, 0 to remove and

In [ ]:
#Import necessary packages

import os
import sys
import pandas as pd
import numpy as np
import networkx as nx
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os
import math
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType 
from graphframes import *

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Start Spark Context
sc = SparkContext().getOrCreate()

In [ ]:
#Read and clean data
count_df = pd.read_csv('/content/drive/MyDrive/Big Data/data_summary/tweet_retweet_counts.csv')

#Only select users which have high tweet count to exclude irrelevant users
high_count_user = count_df[count_df['tweet_retweet_count'] >= 5]['user_id'].values

df_follower = pd.read_csv('/content/drive/MyDrive/Big Data/data_summary/all_followers.csv')
df_follower.dropna(inplace = True)
df_follower.drop_duplicates(inplace = True)
df_follower.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
#Convert all columns data type to int
df_follower['followers'] = df_follower['followers'].astype(int)
df_follower['user_id'] = df_follower['user_id'].astype(int)
df_follower.info()
df_follower.groupby(['user_id']).count().sort_values(['followers'], ascending=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3683243 entries, 0 to 3683242
Data columns (total 2 columns):
 #   Column     Dtype
---  ------     -----
 0   followers  int64
 1   user_id    int64
dtypes: int64(2)
memory usage: 84.3 MB


,followers
user_id,
961387417759567872,5000
715349560491520000,5000
161769014,5000
158253154,5000
152007388,5000
...,...
834246099858759681,1
743041000076156928,1
743258531378278400,1


In [ ]:
#Set number of followers threshold here

df_follower_count = df_follower.groupby(['user_id']).count().sort_values(['followers'], ascending=False)
high_follower = list(df_follower_count[(df_follower_count['followers'] > 2679) & (df_follower_count['followers'] < 5000)].index)

In [ ]:
df_follower_count.describe()

,followers
count,2267.000000
mean,1624.721217
std,1767.487664
min,1.000000
25%,179.500000
50%,813.000000
75%,2679.500000
max,5000.000000


In [ ]:
#Filter df to only include relevant followers (based on threshold set)

get_col_count_df_grouped = count_df[count_df['user_id'].isin(high_follower)].groupby('user_id')
user_id = list(get_col_count_df_grouped.mean().index)
percentage_fake = get_col_count_df_grouped.mean()['percentage_fake'].values
tweet_retweet_count = get_col_count_df_grouped.sum()['tweet_retweet_count'].values

In [ ]:
count_df[count_df['user_id'].isin(high_follower)].groupby('user_id').mean().sort_values(by=['percentage_fake'], ascending=False).describe()

,tweet_real_news_count,tweet_fake_news_count,tweet_count,retweet_real_news_count,retweet_fake_news_count,retweet_count,tweet_retweet_fake_count,tweet_retweet_real_count,tweet_retweet_count,percentage_fake
count,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000
mean,1.591512,1.217507,2.809019,0.411141,0.249337,0.660477,1.466844,2.002653,3.469496,0.434533
std,4.644573,3.800254,6.842112,1.075938,0.547145,1.309277,3.820381,4.770532,6.947734,0.437108
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,0.333333
75%,1.000000,1.000000,2.000000,1.000000,0.000000,1.000000,1.000000,2.000000,3.000000,1.000000
max,60.000000,44.000000,84.000000,15.000000,3.000000,16.000000,44.000000,60.000000,84.000000,1.000000


In [ ]:
#Set percentage threshold for % fake news post ratio for users

high_follower_fake_spreader = count_df[count_df['user_id'].isin(high_follower)].groupby('user_id').mean().sort_values(by=['percentage_fake'], ascending=False)
high_follower_fake_spreader[high_follower_fake_spreader['percentage_fake'] >= 0.333333]

,tweet_real_news_count,tweet_fake_news_count,tweet_count,retweet_real_news_count,retweet_fake_news_count,retweet_count,tweet_retweet_fake_count,tweet_retweet_real_count,tweet_retweet_count,percentage_fake
user_id,,,,,,,,,,
87101015,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
2508050305,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
143048512,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
146518467,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
2439422707,0.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,1.000000
...,...,...,...,...,...,...,...,...,...,...
15144122,2.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0,3.0,0.333333
19401317,3.0,2.0,5.0,1.0,0.0,1.0,2.0,4.0,6.0,0.333333
727328316345208833,2.0,0.0,2.0,0.0,1.0,1.0,1.0,2.0,3.0,0.333333


In [ ]:
new_dict_df = {'user_id' : user_id, 'percentage_fake': percentage_fake, 'tweet_retweet_count' : tweet_retweet_count}
filtered_df = pd.DataFrame.from_dict(new_dict_df)

In [ ]:
#Function to filter based on fake post ratio % and threshold of tweet-retweet count set.

def selected_rows(df, fake_percentage, tweet_retweet_count):
  df = df[df['percentage_fake'] >= fake_percentage]
  df = df[df['tweet_retweet_count'] >= tweet_retweet_count]
  print(len(df['user_id'].values))
  return df['user_id'].values

In [ ]:
selected_user_id = selected_rows(filtered_df, 0.333333, 3)

71


In [ ]:
#Functions to create edges and node lists

def create_edge_df_nodes_df(df, selected_id_list):
  edge_df = df[df['user_id'].isin(selected_id_list)]
  edge_df.drop_duplicates(inplace = True)
  edge_df.rename(columns={'user_id': 'src', 'followers': 'dst'}, inplace = True)
  edge_df.reset_index(inplace = True)
  edge_df.drop(columns=['index'], inplace = True)

  selected_dst = list(edge_df['dst'].unique())
  selected_src = list(edge_df['src'].unique())

  for src in selected_src:
    if src not in selected_dst:
      selected_dst.append(src)

  nodes_dict = {'id' : selected_dst, 'node' : selected_dst}
  nodes_df = pd.DataFrame(nodes_dict)

  return (edge_df, nodes_df)

In [ ]:
edge_df, nodes_df = create_edge_df_nodes_df(df_follower, selected_user_id)
edge_df = edge_df[edge_df['dst'] != edge_df['src']]

In [ ]:
#Export edge and node lists

edge_df.to_csv('edge_df.csv')
nodes_df.to_csv('nodes_df.csv')

In [ ]:
#Prepare node and edges lists
edge_df_set = edge_df[['dst', 'src']]
node_edges = [tuple(x) for x in edge_df_set.to_numpy()]

In [ ]:
#Set graphs with NetworkX
G = nx.from_edgelist(node_edges)

In [ ]:
from networkx.algorithms.components import weakly_connected_components
from networkx.algorithms import community
#from networkx.algorithms.community.centrality import girvan_newman

In [ ]:
#Set communities parameters
communities_kl = community.kernighan_lin_bisection(G)
communities_af = community.asyn_fluidc(G, 200)
communities_al = community.asyn_lpa_communities(G)

In [ ]:
#Check communities created with Kernighan Lin Bisection
node_groups_kl = []
for com in communities_kl:
  node_groups_kl.append(list(com))

2


In [ ]:
#Check communities created with Async Fluid
node_groups_af = []
for com in communities_af:
  node_groups_af.append(list(com))

print(len(node_groups_af))

200


In [ ]:
#Check communities created with Async LPA
node_groups_al = []
for com in communities_al:
  node_groups_al.append(list(com))

print(len(node_groups_al))

71


In [ ]:
#Functions to add community to df

def add_community_to_df(df, community_node_group, community_name):
  community_dict = {}
  df_new = df.copy()
  df_new[community_name] = 0

  for i in range(len(community_node_group)):
    community_dict[i] = community_node_group[i]
  
  for key, value in community_dict.items():
    for select_id in value:
      
      df_new[df_new['id'] == select_id][community_name] = key
      print('je')
  return df_new

In [ ]:
community_dict = {}
for i in range(len(node_groups_kl)):
    community_dict[i] = node_groups_kl[i]

In [ ]:
community_dict = {}
for i in range(len(node_groups_af)):
    community_dict[i] = node_groups_af[i]
  
for key, value in community_dict.items():
  for select_id in value:
    nodes_df[nodes_df['id'] == select_id]['community_af'] = key

In [ ]:
community_dict = {}
for i in range(len(node_groups_al)):
    community_dict[i] = node_groups_al[i]
  
for key, value in community_dict.items():
  for select_id in value:
    nodes_df[nodes_df['id'] == select_id]['community_al'] = key

In [ ]:
#Export as Text
with open('node_groups_kl.txt', 'w') as f:
    for item in node_groups_kl:
        f.write("%s\n" % item)

with open('node_groups_af.txt', 'w') as f:
    for item in node_groups_af:
        f.write("%s\n" % item)

with open('node_groups_al.txt', 'w') as f:
    for item in node_groups_al:
        f.write("%s\n" % item)

with open('node_groups_kl.txt', 'w') as f:
    for item in node_groups_kl:
        f.write("%s\n" % item)

In [ ]:
#Export as Pickle File
import pickle

with open('node_groups_kl.pickle', 'wb') as handle:
    pickle.dump(node_groups_kl, handle, protocol = pickle.HIGHEST_PROTOCOL)

with open('node_groups_kl.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(node_groups_kl == b)

True


In [ ]:
with open('node_groups_al.pickle', 'wb') as handle:
    pickle.dump(node_groups_al, handle, protocol = pickle.HIGHEST_PROTOCOL)

with open('node_groups_al.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(node_groups_al == b)

True


In [ ]:
with open('node_groups_af.pickle', 'wb') as handle:
    pickle.dump(node_groups_af, handle, protocol = pickle.HIGHEST_PROTOCOL)

with open('node_groups_af.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(node_groups_af == b)

True
